In [1]:
using Revise
using DataFrames
using XLSX
photochemistry_source_dir = "$(@__DIR__)/Photochemistry/src/"
push!(LOAD_PATH, photochemistry_source_dir)
using Photochemistry  # custom module

┌ Info: Precompiling Photochemistry [top-level]
└ @ Base loading.jl:1423


In [2]:
include("CONSTANTS.jl");

In [3]:
function escape_velocity()
    #=
    Returns escape velocity for the present planet defined in CONSTANTS.jl in m/s.
    =#
    return sqrt(2 * bigG * M_P / R_P) / 100 # the / 100 is to convert from cm to m.
end

escape_velocity()

10359.311697355919

In [31]:

kJ_to_eV(kj) = kj * 6.242e21

function escape_energy(z)
    #=
    Calculates escape energy from planet P for a molecule containing z protons.
    =#
    
    kJ_to_eV((0.5 * z * (1.67e-27 #=kg=#) * (escape_velocity() #=m/s=#)^2) / 1000 #=kJ/J=#)
end

ev_per_molecule(eV_per_mole) = eV_per_mole/6.022e23

function enthalpy_of_reaction(reactants, products, enthalpy_dict)
    
    dfH_products = 0
    dfH_reactants = 0

    for p in products
        dfH_products += enthalpy_dict[p]
    end
    for r in reactants
        dfH_reactants += enthalpy_dict[r]
    end

    # Calculate the total enthalpy of reaction:
    enthalpy_of_rxn = dfH_products - dfH_reactants
    
    # this block adds up the total mass that needs to escape, allows for reactions that produce, e.g., both H and D
    m = 0 
    flag = 0
    if :H in products
        m += 1
        flag += 1
    end
    if :H2 in products
        m += 2
        flag += 1
    end
    if :D in products
        m += 2
        flag += 1
    end
    if :HD in products
        m += 3
        flag += 1
    end
        
    # Convert the total enthalpy of reaction to eV:
    total_reaction_enthalpy_ev = ev_per_molecule(kJ_to_eV(-1*(enthalpy_of_rxn)))
    
    # Calculate how much energy goes into escaping the hot atoms:
    energy_required_to_escape_all_hot = escape_energy(m)
    
    # Calculate how much excess energy we have
    excess_energy = round(total_reaction_enthalpy_ev - energy_required_to_escape_all_hot, digits=2)

    # Calculate whether endothermic or exothermic
    endo_exo = excess_energy > 0 ? "exothermic" : "endothermic"
    
    println("Reaction $(format_chemistry_string(reactants, products))")
    if flag > 1
        println("Flag! Reaction produces two hot atoms")
    end
    println("Raw enthalpy is $(total_reaction_enthalpy_ev) eV")
    println("Mass to escape $(m), so need a minimum of $(m*escape_energy(1)) eV")
    println("excess energy $(excess_energy).")
    println("Positive excess energy: $(excess_energy > 0), so reaction is $(endo_exo)")
    println()
    
    return endo_exo, total_reaction_enthalpy_ev, energy_required_to_escape_all_hot, excess_energy
end

function get_product_and_reactant_cols(df)
    # Determine how many reactant and product columns there are 
    possible_Rcols = ["R1", "R2", "R3"]
    possible_Pcols = ["P1", "P2", "P3"]
    rcols = Symbol.(possible_Rcols[possible_Rcols .∈ Ref(names(df))])
    pcols = Symbol.(possible_Pcols[possible_Pcols .∈ Ref(names(df))])
    
    return rcols, pcols
end

function calculate_enthalpies(df; species=[:H, :D, :H2, :HD], new_cols=nothing, insert_i=nothing)
    #=
    df: A dataframe containing the contents of a reaction spreadsheet to which we must add excess energies
    
    insert_i here should be the first integer at which we will begin inserting columns.
    =#
    
    #Enthalpy of formation 
    enthalpy_df = DataFrame(XLSX.readtable("./Resources/Enthalpies_of_Formation.xlsx", "enthalpy"))
    
    enthalpy = Dict([Symbol(k)=>df_lookup(enthalpy_df, "Species", k, "Enthalpy")[1] for k in enthalpy_df."Species"])

    
    if new_cols != nothing
        j = 0
        for name in new_cols
            insertcols!(df, insert_i+j, "$(name)"=>["" for x in collect(1:size(df)[1])])
            j += 1
        end
    end
    
    rcols, pcols = get_product_and_reactant_cols(df)
    
    # Count endothermic and exothermic
    endo_count = 0
    exo_count = 0
    
    for row in eachrow(df)
        reactants = [Symbol(row.:($r)) for r in rcols]
        products = filter!(x->x!=:none, [Symbol(row.:($p)) for p in pcols])
        
        # Ignore reactions for which we don't have enthalpies of a species:
        if any(x->!(x in keys(enthalpy)), union(reactants, products))
            continue
        end

        if any(x->x in products, species)
            exo_or_endo, total_enthalpy, loss_energy, excess_energy = enthalpy_of_reaction(reactants, products, enthalpy)
            row.rxnEnthalpy = total_enthalpy
            row.totalEscE = loss_energy
            row.excessE = excess_energy
            
            if exo_or_endo == "exothermic"
                exo_count += 1
                row.NTEscape = "Yes"

                if :D in products
                    row.hotD = "Yes"
                end
                if :HD in products
                    row.hotHD = "Yes"
                end
                
                if :H in products
                    row.hotH = "Yes"
                end
                if :H2 in products
                    row.hotH2 = "Yes"
                end
            else 
                # println("$(format_chemistry_string(reactants, products)) is endothermic")
                # println()
                endo_count += 1
                row.NTEscape = "No"
                if :D in products
                    row.hotD = ""
                end
                if :HD in products
                    row.hotHD = ""
                end
                
                if :H in products
                    row.hotH = ""
                end
                if :H2 in products
                    row.hotH2 = ""
                end
            end
        end
    end
    
    println("Exothermic: $(exo_count), Endothermic: $(endo_count)")
    return df
end

function modify_rxn_spreadsheet(spreadsheet; spc=nothing, new_cols=nothing, insert_i=nothing)
    #=
    insert_i is a list of integers at which to insert columns. The order in this list corresponds to the order of sheets in the workbook.
    =#
    xf = XLSX.readxlsx(spreadsheet)
    original_sheets = XLSX.sheetnames(xf)
    
    new_file = "REACTION_NETWORK_VENUS.xlsx"   # "MOLEC_NT_$(spreadsheet)"#
    
    @assert spc != nothing
    
    # Go through each available sheet and open it as a dataframe for modification
    for (j, sheet) in enumerate(original_sheets)
        df = DataFrame(XLSX.readtable(spreadsheet, sheet));
               
        # Now process:        
        if sheet in ["Ion reactions", "Photodissociation", "Photoionization"]
            # Replace any missing values so we don't get rid of all our reactions before processing:
            rcols, pcols = get_product_and_reactant_cols(df)
            for r in rcols
                replace!(df.:($r), missing=>"none")
            end
            for p in pcols
                replace!(df.:($p), missing=>"none")
            end
            
            if insert_i != nothing
                df_to_write = calculate_enthalpies(df; species=spc, new_cols=new_cols, insert_i=insert_i[j])
            else 
                df_to_write = calculate_enthalpies(df; species=spc)
            end
        else
            df_to_write = df
        end
        println()
        log_reactions(df_to_write, sheet, new_file)
    end
end



modify_rxn_spreadsheet (generic function with 1 method)

In [40]:
# NOT USED but left here to easily see
# esc_in_ev_per_species = Dict(:H=>escape_energy(1), :D=>escape_energy(2), :H2=>escape_energy(2), :HD=>escape_energy(3))
    

Dict{Symbol, Float64} with 4 entries:
  :H  => 0.559335
  :D  => 1.11867
  :H2 => 1.11867
  :HD => 1.67801

In [32]:
modify_rxn_spreadsheet("REACTION_NETWORK_VENUS_start.xlsx"; spc=[:H2, :HD, :H, :D])#, new_cols=["rxnEnthalpy", "totalEscE"], insert_i=[0,7,8,8,0,0])


Reaction ArDpl + H2 --> ArHpl + HD
Raw enthalpy is 0.11795742278312965 eV
Mass to escape 3, so need a minimum of 1.678005174369917 eV
excess energy -1.56.
Positive excess energy: false, so reaction is endothermic

Reaction Arpl + H2 --> ArHpl + H
Raw enthalpy is -12.591799402191961 eV
Mass to escape 1, so need a minimum of 0.5593350581233056 eV
excess energy -13.15.
Positive excess energy: false, so reaction is endothermic

Reaction Arpl + HD --> ArDpl + H
Raw enthalpy is -12.70975682497509 eV
Mass to escape 1, so need a minimum of 0.5593350581233056 eV
excess energy -13.27.
Positive excess energy: false, so reaction is endothermic

Reaction Arpl + HD --> ArHpl + D
Raw enthalpy is -12.647772168714711 eV
Mass to escape 2, so need a minimum of 1.1186701162466113 eV
excess energy -13.77.
Positive excess energy: false, so reaction is endothermic

Reaction CHpl + H --> Cpl + H2
Raw enthalpy is 0.38869976751909663 eV
Mass to escape 2, so need a minimum of 1.1186701162466113 eV
excess energy